<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [12]:
!pip install folium pandas

Defaulting to user installation because normal site-packages is not writeable


In [13]:
import folium
import pandas as pd

In [14]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [17]:
## Task 1: Mark all launch sites on a map

# We assume the columns are 'Launch Site', 'Lat', and 'Long'
launch_sites = spacex_df[['Launch Site', 'Lat', 'Long']]

# Step 3: Create a Folium map centered around the average coordinates
center_lat = launch_sites['Lat'].mean()
center_lon = launch_sites['Long'].mean()
site_map = folium.Map(location=[center_lat, center_lon], zoom_start=7)

# Step 4: Add markers for each launch site
for _, site in launch_sites.iterrows():
    folium.Marker(
        location=[site['Lat'], site['Long']],
        popup=site['Launch Site']
    ).add_to(site_map)

# Step 5: Display the map (in Jupyter Notebook, use this to display the map)
site_map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [18]:
# Download and read the `spacex_launch_geo.csv`
import pandas as pd
import requests
import io

# Step 1: Download the CSV file
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
response = requests.get(url)

# Step 2: Load the CSV content into a pandas DataFrame
spacex_csv_file = io.StringIO(response.text)
spacex_df = pd.read_csv(spacex_csv_file)

# Step 3: Display the DataFrame
spacex_df.head()


,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Now, you can take a look at what are the coordinates for each site.


In [19]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [20]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)
site_map

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [21]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [22]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for _, site in launch_sites.iterrows():
    # Get the coordinates of the launch site
    coordinate = [site['Lat'], site['Long']]
    
    # Add a Circle for the launch site
    folium.Circle(
        location=coordinate,
        radius=1000,  # Circle radius in meters
        color='#000000',  # Circle border color
        fill=True,
        fill_color='#3186cc',  # Circle fill color
        fill_opacity=0.4  # Circle opacity
    ).add_child(folium.Popup(site['Launch Site'])).add_to(site_map)
    
    # Add a Marker for the launch site with a label
    folium.Marker(
        location=coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12; color:#d35400;"><b>{site["Launch Site"]}</b></div>'
        )
    ).add_to(site_map)

# Display the map (this works in Jupyter Notebook)
site_map   

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [41]:
# Task 2: Mark the success/failed launches for each site on the map
import folium

# Menentukan lokasi awal peta berdasarkan data pertama dalam dataset
initial_location = [spacex_df['Lat'].mean(), spacex_df['Long'].mean()]

spacex_map = folium.Map(location=initial_location, zoom_start=10)

# Menampilkan peta
spacex_map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [34]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [35]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [37]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red


# Menambahkan marker yang menandai keberhasilan/kegagalan
for index, launch_site in spacex_df.iterrows():
    # Menentukan warna marker berdasarkan kolom 'class'
    if launch_site['class'] == 1:  # 1 = Berhasil
        marker_color = 'green'
    else:  # 0 = Gagal
        marker_color = 'red'
    
    # Menambahkan marker dengan warna yang sesuai
    folium.Marker(
        location=[launch_site['Lat'], launch_site['Long']],
        popup=f"Launch Site: {launch_site['Launch Site']}\nSuccess: {launch_site['class']}",
        icon=folium.Icon(color=marker_color)
    ).add_to(spacex_map)

# Menampilkan peta
spacex_map

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [44]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
# Fungsi untuk menentukan warna marker berdasarkan nilai di kolom 'class'
def get_marker_color(launch_class):
    return 'green' if launch_class == 1 else 'red'

# Membuat kolom baru 'marker_color' berdasarkan nilai di kolom 'class'
spacex_df['marker_color'] = spacex_df['class'].apply(get_marker_color)

# Menentukan lokasi awal peta
initial_location = [spacex_df['Lat'].mean(), spacex_df['Long'].mean()]
site_map = folium.Map(location=initial_location, zoom_start=5)

# Membuat MarkerCluster
marker_cluster = MarkerCluster().add_to(site_map)

for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
        # Membuat Marker dengan koordinat dan ikon yang sesuai
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        popup=f"Launch Site: {record['Launch Site']}\nSuccess: {record['class']}",
        icon=folium.Icon(color='white', icon_color=record['marker_color'])  # Customize icon
    )
    marker_cluster.add_child(marker)

site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [49]:
# TASK 3: Calculate the distances between a launch site to its proximities
from math import radians, sin, cos, sqrt, atan2

# Haversine function to calculate the distance between two lat/lon points
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in kilometers
    
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Compute differences
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    # Distance in kilometers
    distance = R * c
    return distance

# Example coordinates: Launch site and a proximity (replace with actual data)
launch_site_coords = (28.562302, -80.577356)  # Example: Kennedy Space Center
proximity_coords = (28.5721, -80.6493)  # Example: Closest airport (Melbourne, FL)

# Calculate the distance
distance = haversine(launch_site_coords[0], launch_site_coords[1], proximity_coords[0], proximity_coords[1])
print(f"Distance: {distance:.2f} km")


Distance: 7.11 km


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [50]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [51]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [52]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
# Create a map centered around the launch site
launch_site_lat = 28.562302
launch_site_lon = -80.577356

m = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=12)

# Add MousePosition to track coordinates
mouse_position = MousePosition(
    position='topright',
    separator=', ',
    prefix="Lat, Lon",
    lat_formatter="function(num) {return L.Util.formatNum(num, 5);}",
    lng_formatter="function(num) {return L.Util.formatNum(num, 5);}"
)
m.add_child(mouse_position)

# Display map
m

In [54]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

import folium
from folium.plugins import MousePosition
from folium.features import DivIcon

# Haversine function to calculate the distance between two lat/lon points
def haversine(lat1, lon1, lat2, lon2):
    from math import radians, sin, cos, sqrt, atan2
    R = 6371.0  # Earth radius in kilometers
    
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Compute differences
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    # Distance in kilometers
    distance = R * c
    return distance

# Coordinates of the launch site and coastline
launch_site_lat = 28.562302
launch_site_lon = -80.577356

# Example coordinates for closest coastline (replace with actual values)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate the distance between launch site and coastline
distance_coastline = haversine(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create a folium map centered around the launch site
m = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=12)

# Add a marker for the coastline point, displaying the distance
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance_coastline),
    )
)

# Add the marker to the map
distance_marker.add_to(m)

# Add a MousePosition plugin to track coordinates
mouse_position = MousePosition(
    position='topright',
    separator=', ',
    prefix="Lat, Lon",
    lat_formatter="function(num) {return L.Util.formatNum(num, 5);}",
    lng_formatter="function(num) {return L.Util.formatNum(num, 5);}"
)
m.add_child(mouse_position)

# Display the map
m


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [55]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
#lines=folium.PolyLine(locations=coordinates, weight=1)
#site_map.add_child(lines)

import folium
from folium.plugins import MousePosition
from folium.features import DivIcon

# Haversine function to calculate the distance between two lat/lon points
def haversine(lat1, lon1, lat2, lon2):
    from math import radians, sin, cos, sqrt, atan2
    R = 6371.0  # Earth radius in kilometers
    
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Compute differences
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    # Distance in kilometers
    distance = R * c
    return distance

# Coordinates of the launch site and coastline
launch_site_lat = 28.562302
launch_site_lon = -80.577356

# Example coordinates for closest coastline (replace with actual values)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate the distance between launch site and coastline
distance_coastline = haversine(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Create a folium map centered around the launch site
m = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=12)

# Add a marker for the coastline point, displaying the distance
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance_coastline),
    )
)
distance_marker.add_to(m)

# Add a PolyLine between the launch site and coastline
coordinates = [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]]
lines = folium.PolyLine(locations=coordinates, weight=1, color='blue')

# Add the PolyLine to the map
m.add_child(lines)

# Add MousePosition plugin to track coordinates
mouse_position = MousePosition(
    position='topright',
    separator=', ',
    prefix="Lat, Lon",
    lat_formatter="function(num) {return L.Util.formatNum(num, 5);}",
    lng_formatter="function(num) {return L.Util.formatNum(num, 5);}"
)
m.add_child(mouse_position)

# Display the map
m


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [56]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site

import folium
from folium.features import DivIcon

# Haversine function to calculate distance between two lat/lon points
def haversine(lat1, lon1, lat2, lon2):
    from math import radians, sin, cos, sqrt, atan2
    R = 6371.0  # Earth radius in kilometers
    
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Compute differences
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    # Distance in kilometers
    distance = R * c
    return distance

# Coordinates of the launch site
launch_site_lat = 28.562302
launch_site_lon = -80.577356

# Example coordinates for the closest city, railway, and highway
closest_city = [28.3852, -81.5639]  # Example: Orlando, FL
closest_railway = [28.5623, -80.5707]  # Example: Nearby railway
closest_highway = [28.5641, -80.5852]  # Example: Nearby highway

# Calculate distances from the launch site
distance_city = haversine(launch_site_lat, launch_site_lon, closest_city[0], closest_city[1])
distance_railway = haversine(launch_site_lat, launch_site_lon, closest_railway[0], closest_railway[1])
distance_highway = haversine(launch_site_lat, launch_site_lon, closest_highway[0], closest_highway[1])

# Create a folium map centered around the launch site
m = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=12)

# Add a marker for the closest city with the distance
city_marker = folium.Marker(
    closest_city,
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance_city),
    )
)
city_marker.add_to(m)

# Add a marker for the closest railway with the distance
railway_marker = folium.Marker(
    closest_railway,
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance_railway),
    )
)
railway_marker.add_to(m)

# Add a marker for the closest highway with the distance
highway_marker = folium.Marker(
    closest_highway,
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance_highway),
    )
)
highway_marker.add_to(m)

# Draw PolyLines between the launch site and each point
folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], closest_city], color='blue', weight=2).add_to(m)
folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], closest_railway], color='green', weight=2).add_to(m)
folium.PolyLine(locations=[[launch_site_lat, launch_site_lon], closest_highway], color='red', weight=2).add_to(m)

# Display the map
m

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
